In [ ]:
import ast
import boto3
import firebase_admin
import json
import numpy as np
import os
import pandas as pd
import requests
import time
import torch
import torch.nn.functional as F
import vertexai
from botocore.exceptions import ClientError
from decimal import Decimal
from dotenv import load_dotenv
from firebase_admin import credentials, firestore
from google.cloud import storage
from google.cloud import aiplatform
from requests.exceptions import ReadTimeout
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
from vertexai.language_models import TextEmbeddingModel

from google.cloud.aiplatform.matching_engine import MatchingEngineIndexConfig


In [3]:
# uploading the environment variables and get the API key
load_dotenv()
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")

# Asegurar que Vertex AI use las credenciales correctas
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../../bubbo-dfba0-47e395cdcdc7.json"


In [20]:
from google.cloud import aiplatform

PROJECT_ID = "bubbo-dfba0"
REGION = "europe-southwest1"
INDEX_DISPLAY_NAME = "all-content-embeddings-indexes"
GCS_BUCKET_URI = "gs://embeddings_bucket_backup"

# Inicializar Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION)

# Configuración del índice
config = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=INDEX_DISPLAY_NAME,
    contents_delta_uri=GCS_BUCKET_URI,  # Ruta del bucket en GCS con los archivos JSON
    dimensions=768,  # Asegúrate de que el vector tenga 768 dimensiones
    approximate_neighbors_count=500,  # Número de vecinos aproximados
    distance_measure_type="DOT_PRODUCT_DISTANCE",  # Tipo de medida de distancia
)

print(f"Índice creado: {config.resource_name}")


Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/75629471929/locations/europe-southwest1/indexes/3340439470485798912/operations/6068935848928215040


FailedPrecondition: 400 There are invalid records under files [gs://embeddings_bucket_backup/10017730.json, gs://embeddings_bucket_backup/100439.json, gs://embeddings_bucket_backup/1004375.json, gs://embeddings_bucket_backup/1001430.json, gs://embeddings_bucket_backup/10003669.json, gs://embeddings_bucket_backup/10000125.json, gs://embeddings_bucket_backup/1000646.json, gs://embeddings_bucket_backup/1001721.json, gs://embeddings_bucket_backup/10000963.json, gs://embeddings_bucket_backup/10014.json, gs://embeddings_bucket_backup/1003625.json, gs://embeddings_bucket_backup/1003807.json, gs://embeddings_bucket_backup/10031339.json, gs://embeddings_bucket_backup/10014188.json, gs://embeddings_bucket_backup/10048447.json, gs://embeddings_bucket_backup/1001519.json, gs://embeddings_bucket_backup/10003272.json, gs://embeddings_bucket_backup/1001127.json, gs://embeddings_bucket_backup/1002522.json, gs://embeddings_bucket_backup/100209.json, gs://embeddings_bucket_backup/1001516.json, gs://embe

In [ ]:
# guardo esto como ejemplo de consulta:
# Diccionario de IDs basado en el bucket GCS
id_map = {
    0: "file_001.json",
    1: "file_002.json",
    2: "file_003.json",
    # Agrega más archivos según los embeddings subidos
}

# Supongamos que Matching Engine devuelve estos índices de vecinos
neighbors = [
    {"neighbor_id": 1, "distance": 0.04},
    {"neighbor_id": 0, "distance": 0.07}
]

# Convertimos los índices en IDs reales
real_neighbors = [{"id": id_map[n["neighbor_id"]], "distance": n["distance"]} for n in neighbors]

print(real_neighbors)
